In [1]:
import numpy as np
import pandas as pd
import os
import sklearn.mo

In [2]:
def acquire_mob_sales():
    '''
    This function will read in tables from 3 xlsm excel documents containing sales
    order history for customers
    '''
    # get sales order history for customers
    sales = retention_18 = pd.read_excel('mob_sales.xlsx', sheet_name=0)
    sorder_details = raw_6 = pd.read_excel('mob_sales.xlsx', sheet_name=1)
    # get a customer list
    customer_list = retention_17 = pd.read_excel('mob_sales.xlsx', sheet_name=3)
    # return the 4 dataframes
    return sales, sorder_details, customer_list

In [3]:
def acquire_mob_item_history():
    '''
    This function will retreive a table from an xlsm excel document containing
    item sales history by week for all products
    '''
    # get item sales history
    item_sales = pd.read_excel('mob_sales.xlsx', sheet_name=2)
    # return dataframe
    return item_sales

In [4]:
def prepare_mob_item_history(all_sales_history):
    '''
    This function will take in item sales history for all products, it will then
    drop unnecessary columns and change the change the index to datetime format,
    it will then change the sku names of products to generic numbers.
    '''
    # remove items that are inactive products
    sales_history = all_sales_history[all_sales_history.Forecast == True
                  # remove unnecessary columns
                 ].drop(columns=['Lifetime', 'Forecast'
                                 # remove extra columns that have no data
                                ]).dropna(axis=1
                                          # change the column headers to item_sku
                                         ).set_index('SKU').T
    # convert the index to datetime
    sales_history.index = pd.to_datetime(sales_history.index)
    # resample the index to standard week format
    sales_history = sales_history.resample('W').sum()
    # replace product skus with generic numbers for infosec
    sales_history.columns = ['prod_' + str(x) for x in range(0,len(sales_history.columns))]
    # return the prepared dataframe
    return sales_history

In [5]:
def prepare_mob_sales_order(sales, sorder_details, customer_list):
    '''
    This function will take in dataframes of sales orders, the order details, and a list
    of customers. it will then merge the dfs, fill in null values with appropriate values,
    remove suspended customers, and then remove some now unnecessary info.
    '''
    # merge sales orders with the sales order details
    sales_orders = pd.merge(left=sales, 
                            right=sorder_details, 
                            how='left', 
                            on='OrderID')
    # fill na values
    sales_orders.QtyOrdered.fillna(1, inplace=True)
    sales_orders.QtyShipped.fillna(0, inplace=True)
    # get a list of suspended customers from the customer list, since we don't want
    # suspended customers in the final report
    suspended_list = customer_list[['CustomerID', 'Suspended'
                               ]][customer_list.Suspended == False]
    # merge the suspended list with the sales orders
    sorders = pd.merge(left=sales_orders, 
                       right=suspended_list, 
                       how='left', 
                       on='CustomerID')
    # suspended customers will ahve a null value in the Suspended column,
    # drop the suspended info then remove the suspended column
    sorders = sorders[sorders.Suspended.isna() == False].drop(columns='Suspended')
    # change column names to lowercase
    sorders.columns = [col.lower() for col in sorders]
    # set the index to the orderdate so we can work with the data as a time series problem
    sorders = sorders.set_index('orderdate').sort_index()
    # remove the one row that has decimal qty ordered and qty_shipped
    sorders = sorders[~(sorders.qtyordered.astype(int) != sorders.qtyordered)]
    # convert the float type columns to int
    sorders.qtyordered = sorders.qtyordered.astype(int)
    sorders.qtyshipped = sorders.qtyshipped.astype(int)
    # change column names to be more readable
    cols = ['order_id', 'order_no', 'customer_id', 'order_status', 'order_amount',
         'seq', 'qty_ordered', 'qty_shipped', 'item_id']
    sorders.columns = cols
    # return the prepared sales orders dataframe
    return sorders

In [6]:
def wrangle_mob_item_sales():
    '''
    This function will check for existance of item sales history csv file in the local
    directory, if one does not exist it will acquire the dataset, prepare it, then 
    create a csv file and either way it will return the prepared dataframe.
    '''
    # check for existance of item_history.csv file in the local directory
    if os.path.exists('item_history.csv'):
        # read in csv file if one exists
        item_history = pd.read_csv('item_history.csv', index_col=0)
    # if csv file does not exist
    else:
        # read in dataset from excel file
        all_item_history = acquire_mob_item_history()
        # prepare the data
        item_history = prepare_mob_item_history(all_item_history)
        # write a new csv file to the local directory
        item_history.to_csv('item_history.csv')
    # return the prepared dataframe
    return item_history

In [7]:
def wrangle_mob_sales():
    '''
    This function will check for existance of sales orders history csv file in the local
    directory, if one does not exist it will acquire the dataset, prepare it, then 
    create a csv file and either way it will return the prepared dataframe.
    '''
    # check for existance of item_history.csv file in the local directory
    if os.path.exists('sales_history.csv'):
        # read in csv file if one exists
        sales = pd.read_csv('sales_history.csv', index_col=0)
    # if csv file does not exist
    else:
        # read in dataset from excel file
        sales_history, sorder_details, customer_list = acquire_mob_sales()
        # prepare the data
        sales = prepare_mob_sales_order(sales_history, sorder_details, customer_list)
        # write a new csv file to the local directory
        sales.to_csv('sales_history.csv')
    # return the prepared dataframe
    return sales

In [8]:
items = wrangle_mob_item_sales()

In [9]:
items

,prod_0,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,...,prod_102,prod_103,prod_104,prod_105,prod_106,prod_107,prod_108,prod_109,prod_110,prod_111
2015-01-04,0,0,1,0,0,0,0,25,24,205,...,0,0,0,0,0,0,0,0,0,0
2015-01-11,2,0,50,0,0,0,0,24,30,145,...,0,0,0,0,0,0,0,0,0,0
2015-01-18,1,2,150,0,0,0,0,48,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-25,0,1,100,0,0,0,0,0,55,315,...,0,0,0,0,0,0,0,0,0,0
2015-02-01,0,2,17,0,0,0,0,71,8,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,0,0,0,5,0,5,0,0,0,115,...,0,0,0,0,11,0,0,0,0,1
2019-11-03,1,98,1,11,24,0,0,0,0,0,...,0,20,0,0,7,0,0,1,0,1
2019-11-10,4,3,26,5,0,-1,1,23,28,0,...,0,8,0,0,1,0,0,0,0,0
2019-11-17,1,1,36,4,1,4,1,0,26,115,...,2,12,0,0,23,0,0,1,0,0


In [91]:
sales = wrangle_mob_sales()

In [92]:
sales

,orderid,orderno,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
orderdate,,,,,,,,,
2016-05-10,26,SO035,666,3,603.84,1,96.0,96.0,1358
2016-05-10,27,SO036,666,3,138.38,1,22.0,22.0,1358
2016-05-17,31,SO033,666,3,10.29,1,1.0,1.0,1359
2016-05-17,32,SO037,666,3,1183.35,1,115.0,115.0,1360
2016-05-17,33,SO041,666,3,143.38,1,22.0,22.0,1358
...,...,...,...,...,...,...,...,...,...
2020-01-03,22034,M32047,13,3,107.98,1,1.0,1.0,11173
2020-01-04,22321,32334,13,3,1068.45,1,17.0,17.0,10802
2020-01-05,22897,32910,13,3,446.72,1,7.0,7.0,10802


In [12]:
sales, sorder_details, customer_list = acquire_mob_sales()

In [13]:
sales.head()

,OrderID,OrderNo,OrderDate,CustomerID,OrderStatus,OrderAmount
0,37014,46210,2019-08-20,0,2,44.99
1,37015,46211,2019-08-21,0,2,109.98
2,37016,46212,2019-08-22,0,2,64.99
3,37017,46213,2019-08-23,0,2,79.98
4,37018,46214,2019-08-24,0,2,44.99


In [14]:
all_sales_history = acquire_mob_item_history()

In [15]:
all_sales_history.head()

,SKU,Forecast,Lifetime,2014-12-29 00:00:00,2015-01-05 00:00:00,2015-01-12 00:00:00,2015-01-19 00:00:00,2015-01-26 00:00:00,2015-02-02 00:00:00,2015-02-09 00:00:00,...,2020-09-28 00:00:00,2020-10-05 00:00:00,2020-10-12 00:00:00,2020-10-19 00:00:00,2020-10-26 00:00:00,2020-11-02 00:00:00,2020-11-09 00:00:00,2020-11-16 00:00:00,2020-11-23 00:00:00,2020-11-30 00:00:00
0,1146988.0,False,52,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1162753.0,False,39,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1162754.0,False,4,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1162755.0,False,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1162756.0,False,6,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
all_sales_history.T

,0,1,2,3,4,5,6,7,8,9,...,498,499,500,501,502,503,504,505,506,507
SKU,1146988.0,1162753.0,1162754.0,1162755.0,1162756.0,1162757.0,1162758.0,1162759.0,1162760.0,1162761.0,...,1536340.068182,1537009.798419,1537679.528656,1538349.258893,1539018.98913,1539688.719368,1540358.449605,1541028.179842,1541697.910079,1542367.640316
Forecast,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,True,True,False,False
Lifetime,52,39,4,1,6,4,35,10,18,14,...,5,67,1,4,53,178,11,35,49,7
2014-12-29 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-05 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
sale_1 = all_sales_history[['SKU', 'Forecast', 'Lifetime']]

In [18]:
sale_1

,SKU,Forecast,Lifetime
0,1.146988e+06,False,52
1,1.162753e+06,False,39
2,1.162754e+06,False,4
3,1.162755e+06,False,1
4,1.162756e+06,False,6
...,...,...,...
503,1.539689e+06,True,178
504,1.540358e+06,True,11
505,1.541028e+06,True,35
506,1.541698e+06,False,49


In [19]:
sale_1 = sale_1.drop(columns='Lifetime')

In [20]:
sale_2 = all_sales_history.drop(
    columns=['SKU', 'Forecast', 'Lifetime'])

In [21]:
sale_2 = sale_2.dropna(axis=1)

In [22]:
sale_2.isna().sum()

2014-12-29    0
2015-01-05    0
2015-01-12    0
2015-01-19    0
2015-01-26    0
             ..
2019-10-21    0
2019-10-28    0
2019-11-04    0
2019-11-11    0
2019-11-18    0
Length: 256, dtype: int64

In [23]:
all_sales_history[all_sales_history.Forecast == True
                 ].drop(columns=['Lifetime', 'Forecast'
                                ]).dropna(axis=1
                                         ).set_index('SKU').head().T

SKU,1.245007e+06,1.245677e+06,1.246347e+06,1.249696e+06,1.250365e+06
2014-12-29,0,0,1,0,0
2015-01-05,2,0,50,0,0
2015-01-12,1,2,150,0,0
2015-01-19,0,1,100,0,0
2015-01-26,0,2,17,0,0
...,...,...,...,...,...
2019-10-21,0,0,0,5,0
2019-10-28,1,98,1,11,24
2019-11-04,4,3,26,5,0
2019-11-11,1,1,36,4,1


In [24]:
sales_history = all_sales_history[all_sales_history.Forecast == True
                 ].drop(columns=['Lifetime', 'Forecast'
                                ]).dropna(axis=1
                                         ).set_index('SKU').T

In [25]:
sales_history

SKU,1.245007e+06,1.245677e+06,1.246347e+06,1.249696e+06,1.250365e+06,1.285191e+06,1.287870e+06,1.288540e+06,1.289210e+06,1.289879e+06,...,1.531652e+06,1.532322e+06,1.532991e+06,1.533661e+06,1.535001e+06,1.535670e+06,1.536340e+06,1.539689e+06,1.540358e+06,1.541028e+06
2014-12-29,0,0,1,0,0,0,0,25,24,205,...,0,0,0,0,0,0,0,0,0,0
2015-01-05,2,0,50,0,0,0,0,24,30,145,...,0,0,0,0,0,0,0,0,0,0
2015-01-12,1,2,150,0,0,0,0,48,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-19,0,1,100,0,0,0,0,0,55,315,...,0,0,0,0,0,0,0,0,0,0
2015-01-26,0,2,17,0,0,0,0,71,8,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-21,0,0,0,5,0,5,0,0,0,115,...,0,0,0,0,11,0,0,0,0,1
2019-10-28,1,98,1,11,24,0,0,0,0,0,...,0,20,0,0,7,0,0,1,0,1
2019-11-04,4,3,26,5,0,-1,1,23,28,0,...,0,8,0,0,1,0,0,0,0,0
2019-11-11,1,1,36,4,1,4,1,0,26,115,...,2,12,0,0,23,0,0,1,0,0


In [26]:
# sales_history.index = pd.to_datetime(sales_history.drop(
#     columns=['SKU', 'Desc']).index, format='%Y-%m-%d')

In [27]:
sales_history.index

Index([2014-12-29 00:00:00, 2015-01-05 00:00:00, 2015-01-12 00:00:00,
       2015-01-19 00:00:00, 2015-01-26 00:00:00, 2015-02-02 00:00:00,
       2015-02-09 00:00:00, 2015-02-16 00:00:00, 2015-02-23 00:00:00,
       2015-03-02 00:00:00,
       ...
       2019-09-16 00:00:00, 2019-09-23 00:00:00, 2019-09-30 00:00:00,
       2019-10-07 00:00:00, 2019-10-14 00:00:00, 2019-10-21 00:00:00,
       2019-10-28 00:00:00, 2019-11-04 00:00:00, 2019-11-11 00:00:00,
       2019-11-18 00:00:00],
      dtype='object', length=256)

In [28]:
date=pd.to_datetime(sales_history.index)

In [29]:
sales_history.index = pd.to_datetime(sales_history.index)

In [30]:
sales_history = sales_history.resample('W').sum()

In [31]:
sku = ['prod_' + str(x) for x in range(0,len(sales_history.columns))]

In [32]:
sku

['prod_0',
 'prod_1',
 'prod_2',
 'prod_3',
 'prod_4',
 'prod_5',
 'prod_6',
 'prod_7',
 'prod_8',
 'prod_9',
 'prod_10',
 'prod_11',
 'prod_12',
 'prod_13',
 'prod_14',
 'prod_15',
 'prod_16',
 'prod_17',
 'prod_18',
 'prod_19',
 'prod_20',
 'prod_21',
 'prod_22',
 'prod_23',
 'prod_24',
 'prod_25',
 'prod_26',
 'prod_27',
 'prod_28',
 'prod_29',
 'prod_30',
 'prod_31',
 'prod_32',
 'prod_33',
 'prod_34',
 'prod_35',
 'prod_36',
 'prod_37',
 'prod_38',
 'prod_39',
 'prod_40',
 'prod_41',
 'prod_42',
 'prod_43',
 'prod_44',
 'prod_45',
 'prod_46',
 'prod_47',
 'prod_48',
 'prod_49',
 'prod_50',
 'prod_51',
 'prod_52',
 'prod_53',
 'prod_54',
 'prod_55',
 'prod_56',
 'prod_57',
 'prod_58',
 'prod_59',
 'prod_60',
 'prod_61',
 'prod_62',
 'prod_63',
 'prod_64',
 'prod_65',
 'prod_66',
 'prod_67',
 'prod_68',
 'prod_69',
 'prod_70',
 'prod_71',
 'prod_72',
 'prod_73',
 'prod_74',
 'prod_75',
 'prod_76',
 'prod_77',
 'prod_78',
 'prod_79',
 'prod_80',
 'prod_81',
 'prod_82',
 'prod_83',
 '

In [33]:
sales_history.columns = sku

In [34]:
sales_history.columns = ['prod_' + str(x) for x in range(0,len(sales_history.columns))]

In [35]:
sales_history

,prod_0,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,...,prod_102,prod_103,prod_104,prod_105,prod_106,prod_107,prod_108,prod_109,prod_110,prod_111
2015-01-04,0,0,1,0,0,0,0,25,24,205,...,0,0,0,0,0,0,0,0,0,0
2015-01-11,2,0,50,0,0,0,0,24,30,145,...,0,0,0,0,0,0,0,0,0,0
2015-01-18,1,2,150,0,0,0,0,48,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-25,0,1,100,0,0,0,0,0,55,315,...,0,0,0,0,0,0,0,0,0,0
2015-02-01,0,2,17,0,0,0,0,71,8,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,0,0,0,5,0,5,0,0,0,115,...,0,0,0,0,11,0,0,0,0,1
2019-11-03,1,98,1,11,24,0,0,0,0,0,...,0,20,0,0,7,0,0,1,0,1
2019-11-10,4,3,26,5,0,-1,1,23,28,0,...,0,8,0,0,1,0,0,0,0,0
2019-11-17,1,1,36,4,1,4,1,0,26,115,...,2,12,0,0,23,0,0,1,0,0


In [36]:
sales_history.shape

(256, 112)

In [37]:
sales_history.resample('M').sum()

,prod_0,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,...,prod_102,prod_103,prod_104,prod_105,prod_106,prod_107,prod_108,prod_109,prod_110,prod_111
2015-01-31,3,3,301,0,0,0,0,97,109,665,...,0,0,0,0,0,0,0,0,0,0
2015-02-28,4,5,121,0,0,0,0,175,122,575,...,0,0,0,0,0,0,0,0,0,0
2015-03-31,126,112,205,0,0,0,0,424,186,376,...,0,0,0,0,0,0,0,0,0,0
2015-04-30,2,109,253,0,0,0,0,0,91,430,...,0,0,0,0,0,0,0,0,0,0
2015-05-31,51,152,161,0,0,0,0,0,114,620,...,0,0,0,0,0,0,0,0,0,0
2015-06-30,13,0,175,0,0,0,0,63,31,249,...,0,0,0,0,0,0,0,0,0,0
2015-07-31,1,19,155,0,0,0,0,75,82,345,...,0,0,0,0,0,0,0,0,0,0
2015-08-31,15,87,161,0,0,0,0,91,104,345,...,0,0,0,0,0,0,0,0,0,0
2015-09-30,2,214,105,0,0,0,0,64,61,230,...,0,0,0,0,0,0,0,0,0,0
2015-10-31,36,148,159,0,0,0,0,77,89,575,...,0,0,0,0,0,0,0,0,0,0


In [38]:
sales_orders = sales

In [39]:
sales_orders.head()

,OrderID,OrderNo,OrderDate,CustomerID,OrderStatus,OrderAmount
0,37014,46210,2019-08-20,0,2,44.99
1,37015,46211,2019-08-21,0,2,109.98
2,37016,46212,2019-08-22,0,2,64.99
3,37017,46213,2019-08-23,0,2,79.98
4,37018,46214,2019-08-24,0,2,44.99


In [40]:
sorder_details.head()

,OrderID,Seq,QtyOrdered,QtyShipped,ItemID
0,4,1,1.0,NaN,3303
1,17,1,15.0,NaN,2077
2,22,1,1.0,NaN,3303
3,23,1,1.0,NaN,3309
4,26,1,96.0,96.0,1358


In [41]:
sales_orders = pd.merge(left=sales_orders, right=sorder_details, how='left', on='OrderID')

In [42]:
sales_orders[sales_orders.OrderID == 40251]

,OrderID,OrderNo,OrderDate,CustomerID,OrderStatus,OrderAmount,Seq,QtyOrdered,QtyShipped,ItemID
72495,40251,M49447,2019-11-21,23836,2,66.95,1,1.0,1.0,11653
72496,40251,M49447,2019-11-21,23836,2,66.95,2,1.0,1.0,3301
72497,40251,M49447,2019-11-21,23836,2,66.95,3,1.0,1.0,11191
72498,40251,M49447,2019-11-21,23836,2,66.95,4,3.0,3.0,11372


In [43]:
sales_orders.columns = [col.lower() for col in sales_orders]

In [44]:
sales_orders.head()

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
0,37014,46210,2019-08-20,0,2,44.99,1,1.0,NaN,11853
1,37015,46211,2019-08-21,0,2,109.98,1,1.0,NaN,10620
2,37015,46211,2019-08-21,0,2,109.98,2,1.0,NaN,3264
3,37016,46212,2019-08-22,0,2,64.99,1,1.0,NaN,11208
4,37017,46213,2019-08-23,0,2,79.98,1,1.0,NaN,3264


In [45]:
sales_orders.isna().sum()

orderid         0
orderno         0
orderdate       0
customerid      0
orderstatus     0
orderamount     0
seq             0
qtyordered      1
qtyshipped     77
itemid          0
dtype: int64

In [46]:
sales_orders[sales_orders.qtyordered.isna() == True]

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
3439,2619,12619,2016-09-08,213,3,1474.66,18,NaN,1.0,10043


In [47]:
sales_orders[sales_orders.customerid == 213]

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
3390,1113,SO065,2016-06-07,213,3,593.82,1,4.0,4.0,3303
3391,1113,SO065,2016-06-07,213,3,593.82,2,1.0,1.0,3247
3392,1113,SO065,2016-06-07,213,3,593.82,3,1.0,1.0,3249
3393,1113,SO065,2016-06-07,213,3,593.82,4,1.0,1.0,3256
3394,1113,SO065,2016-06-07,213,3,593.82,5,1.0,1.0,3248
...,...,...,...,...,...,...,...,...,...,...
3481,20717,30717,2018-06-07,213,3,561.82,4,2.0,2.0,3270
3482,20717,30717,2018-06-07,213,3,561.82,5,2.0,2.0,3167
3483,27397,37274,2018-11-07,213,3,718.79,1,11.0,11.0,10620
3484,27397,37274,2018-11-07,213,3,718.79,2,10.0,10.0,11191


#### The only null in qtyordered has a qtyshipped of 1, and a completed status (3) so lets fill the qty ordered with 1

In [48]:
sales_orders.qtyordered = sales_orders.qtyordered.fillna(1)

In [49]:
sales_orders[sales_orders.qtyordered.isna() == True]

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid


In [50]:
sales_orders[sales_orders.qtyshipped.isna() == True].orderdate.agg(['min','max'])

min   2016-07-08
max   2019-11-22
Name: orderdate, dtype: datetime64[ns]

In [51]:
sales_orders[(sales_orders.qtyshipped.isna() == True) & 
             (sales_orders.orderdate >= '2019')]

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
0,37014,46210,2019-08-20,0,2,44.99,1,1.0,NaN,11853
1,37015,46211,2019-08-21,0,2,109.98,1,1.0,NaN,10620
2,37015,46211,2019-08-21,0,2,109.98,2,1.0,NaN,3264
3,37016,46212,2019-08-22,0,2,64.99,1,1.0,NaN,11208
4,37017,46213,2019-08-23,0,2,79.98,1,1.0,NaN,3264
5,37017,46213,2019-08-23,0,2,79.98,2,1.0,NaN,3301
6,37018,46214,2019-08-24,0,2,44.99,1,1.0,NaN,11853
7,37019,46215,2019-08-25,0,2,14.99,1,1.0,NaN,11009
8,37020,46216,2019-08-26,0,2,44.99,1,1.0,NaN,10620
979,40258,49454,2019-11-21,64,2,419.80,1,4.0,NaN,11845


#### Lets fill the null values in qtyshipped with 0

In [52]:
sales_orders.qtyshipped.fillna(0, inplace=True)

In [53]:
sales_orders[sales_orders.qtyshipped.isna() == True]

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid


In [54]:
sales_orders.isna().sum()

orderid        0
orderno        0
orderdate      0
customerid     0
orderstatus    0
orderamount    0
seq            0
qtyordered     0
qtyshipped     0
itemid         0
dtype: int64

In [55]:
customer_list.head()

,CustomerID,Suspended,CustomerTypeID
0,2,False,1
1,3,False,1
2,5,True,1
3,6,False,9
4,9,False,1


In [56]:
suspended_list = customer_list[['CustomerID', 'Suspended'
                               ]][customer_list.Suspended == False]

In [57]:
suspended_list

,CustomerID,Suspended
0,2,False
1,3,False
3,6,False
4,9,False
5,11,False
...,...,...
14216,23836,False
14217,23838,False
14218,23839,False
14219,23840,False


In [58]:
suspended_list.columns = [col.lower() for col in suspended_list]

In [59]:
sorders = pd.merge(left=sales_orders, right=suspended_list, how='left', on='customerid')

In [60]:
sorders

,orderid,orderno,orderdate,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid,suspended
0,37014,46210,2019-08-20,0,2,44.99,1,1.0,0.0,11853,NaN
1,37015,46211,2019-08-21,0,2,109.98,1,1.0,0.0,10620,NaN
2,37015,46211,2019-08-21,0,2,109.98,2,1.0,0.0,3264,NaN
3,37016,46212,2019-08-22,0,2,64.99,1,1.0,0.0,11208,NaN
4,37017,46213,2019-08-23,0,2,79.98,1,1.0,0.0,3264,NaN
...,...,...,...,...,...,...,...,...,...,...,...
72497,40251,M49447,2019-11-21,23836,2,66.95,3,1.0,1.0,11191,False
72498,40251,M49447,2019-11-21,23836,2,66.95,4,3.0,3.0,11372,False
72499,40272,49468,2019-11-21,23838,2,67.76,1,1.0,1.0,11038,False
72500,40275,49471,2019-11-22,23839,2,73.99,1,1.0,0.0,3264,False


In [61]:
sorders = sorders[sorders.suspended.isna() == False].drop(columns='suspended')

In [62]:
sorders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69493 entries, 9 to 72501
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   orderid      69493 non-null  int64         
 1   orderno      69493 non-null  object        
 2   orderdate    69493 non-null  datetime64[ns]
 3   customerid   69493 non-null  int64         
 4   orderstatus  69493 non-null  int64         
 5   orderamount  69493 non-null  float64       
 6   seq          69493 non-null  int64         
 7   qtyordered   69493 non-null  float64       
 8   qtyshipped   69493 non-null  float64       
 9   itemid       69493 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(5), object(1)
memory usage: 5.8+ MB


In [63]:
sorders = sorders.set_index('orderdate').sort_index()

In [67]:
sorders.qtyordered.value_counts()

 1.0      36483
 2.0      11463
 3.0       4417
 4.0       3957
 5.0       2450
          ...  
 88.0         1
 205.0        1
-5.0          1
-11.0         1
-9.0          1
Name: qtyordered, Length: 206, dtype: int64

In [70]:
(sorders.qtyordered.astype(int) != sorders.qtyordered).sum()

1

In [72]:
(sorders.qtyshipped.astype(int) != sorders.qtyshipped).sum()

1

In [78]:
sorders[sorders.qtyordered.astype(int) != sorders.qtyordered]

,orderid,orderno,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
orderdate,,,,,,,,,
2018-06-28,21586,RMA31599,15230,3,1302.12,2,0.15,0.15,10474


#### There is only one row that has decimal values in qtyordered or qtyshipped

In [75]:
1302.12 / 15

86.80799999999999

#### I know that this itemid is a product with a price between 40-60 dollars, but which doesn't seem to add up right. So lets drop this row

In [79]:
sorders = sorders[~(sorders.qtyordered.astype(int) != sorders.qtyordered)]

In [80]:
sorders

,orderid,orderno,customerid,orderstatus,orderamount,seq,qtyordered,qtyshipped,itemid
orderdate,,,,,,,,,
2016-05-10,26,SO035,666,3,603.84,1,96.0,96.0,1358
2016-05-10,27,SO036,666,3,138.38,1,22.0,22.0,1358
2016-05-17,31,SO033,666,3,10.29,1,1.0,1.0,1359
2016-05-17,32,SO037,666,3,1183.35,1,115.0,115.0,1360
2016-05-17,33,SO041,666,3,143.38,1,22.0,22.0,1358
...,...,...,...,...,...,...,...,...,...
2020-01-03,22034,M32047,13,3,107.98,1,1.0,1.0,11173
2020-01-04,22321,32334,13,3,1068.45,1,17.0,17.0,10802
2020-01-05,22897,32910,13,3,446.72,1,7.0,7.0,10802


#### Now we can change qtyordered and qtyshipped to int

In [81]:
sorders.qtyordered = sorders.qtyordered.astype(int)
sorders.qtyshipped = sorders.qtyshipped.astype(int)

/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1895/4024257974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorders.qtyordered = sorders.qtyordered.astype(int)
/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1895/4024257974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorders.qtyshipped = sorders.qtyshipped.astype(int)


In [82]:
sorders.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69492 entries, 2016-05-10 to 2020-01-06
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   orderid      69492 non-null  int64  
 1   orderno      69492 non-null  object 
 2   customerid   69492 non-null  int64  
 3   orderstatus  69492 non-null  int64  
 4   orderamount  69492 non-null  float64
 5   seq          69492 non-null  int64  
 6   qtyordered   69492 non-null  int64  
 7   qtyshipped   69492 non-null  int64  
 8   itemid       69492 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 5.3+ MB


In [83]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256 entries, 2015-01-04 to 2019-11-24
Columns: 112 entries, prod_0 to prod_111
dtypes: int64(112)
memory usage: 226.0+ KB


In [85]:
sorders.columns.to_list()

['orderid',
 'orderno',
 'customerid',
 'orderstatus',
 'orderamount',
 'seq',
 'qtyordered',
 'qtyshipped',
 'itemid']

In [86]:
cols = ['order_id', 'order_no', 'customer_id', 'order_status', 'order_amount',
     'seq', 'qty_ordered', 'qty_shipped', 'item_id']

In [87]:
sorders.columns = cols

In [88]:
sorders.columns

Index(['order_id', 'order_no', 'customer_id', 'order_status', 'order_amount',
       'seq', 'qty_ordered', 'qty_shipped', 'item_id'],
      dtype='object')

In [93]:
sorders.index.max()

Timestamp('2020-01-06 00:00:00')